<a href="https://colab.research.google.com/github/StrangeTcy/ru-gpts/blob/master/Finetune_and_generate_RuGPTs_deepspeed_megatron_ivolution__javascript__highmem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount our drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Wed Apr  7 05:29:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Finetune RuGPTs in megatron and deepspeed
How to finetune RuGPTs models with megatron and deepspeed. Example for RuGPT3Small. Note for other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
# !pip3 install transformers==3.5.0
!pip3 install transformers

     |████████████████████████████████| 2.2MB 7.9MB/s 
     |████████████████████████████████| 3.3MB 22.8MB/s 
     |████████████████████████████████| 870kB 44.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=9535893866ecc76fe0d569fb7e9b64c88353e21cc9ffea3845f0278a285b4b4c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
# We might install apex later on, but for now we don't need or use it
!sh setup.sh

In [ ]:
# Git clone my clone of sberbank's ru-gpts
!git clone  https://github.com/StrangeTcy/ru-gpts

In [ ]:
# Install deepspeed which we'll probably need later on
!pip install deepspeed==0.3.7

In [ ]:
# Install packages we'll need if we use sparsify (we mgiht?)
!apt-get install llvm-9-dev
!pip install cpufeature
!pip install triton==0.2.3
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7

In [ ]:
# Is deepspeed working ok?
!ds_report

## Download files

In [ ]:
# !wget -O train.txt https://www.dropbox.com/s/oa3v9c7g9bp40xw/train.txt?dl=0
# !wget -O valid.txt https://www.dropbox.com/s/mworl3ld6r3bg62/valid.txt?dl=0

Convert our dataset

In [ ]:
# !gdown https://drive.google.com/uc?id=1u50bbNb5p9cjlx99YuQm1m9yTOgGDiNO
 

with open("/content/drive/MyDrive/ivolution_dataset/js_script_prechunks.json", "r") as ptj:
  sources = ptj.readlines()

# with open("/content/js_script_prechunks.json", "r") as ptj:
#   sources = ptj.readlines()

from collections import defaultdict
from tqdm import tqdm

chunks = defaultdict(list)
num = 0

for l in tqdm(sources[:1000]):
  
  
  if '+++++\n' in l:
    num +=1
  else:
    chunks[num].append(l)
  
  first_line = chunks.get(num)
  if first_line is not None:
    l_splits = first_line[0].split(" ") 
    # for s in l_splits:
    #   print (f"\t\t========{s}")
  
  # with open("/content/trial.txt", "a+") as f:
  with open("/content/huge_dataset.txt", "a+") as f:
    for j in range(len(chunks)):
      for i in range(max(len(chunks[j]),3)):
        prompt = str(''.join(chunks[j][:i]))
        # a reasonable chunk of code for completeion  
        lng = 5
        completion = str(''.join(chunks[j][i:i+lng]))
        if not (prompt == '' or completion == ''):
          f.write(f"<s>prompt: {prompt}\n completion:{completion}</s>\n")
      first_line = chunks.get(j)
      if first_line is not None:
        l_splits = first_line[0].split(" ") 
        for s in range(len(l_splits)):
          # print (f"\t\t========{l_splits[s]}")
          prompt = str(" ".join(l_splits[:s]))
          completion =  str(" ".join(l_splits[s:]))
          if not (prompt == '' or completion == ''):
            f.write(f"<s>Prompt: {prompt}\nCompletion: {completion}</s>\n")
          

In [ ]:
# Get our dataset
# !gdown https://drive.google.com/file/d/1-0K7OdvgX9AuPbJakCaAWgpL5qCSyuB9
# !cp -v /content/drive/Shareddrives/Machine_/ivolution_dataset/huge_dataset.txt /content/trial.txt
# !wc -l /content/drive/Shareddrives/Machine_/ivolution_dataset/huge_dataset.txt
# !wc -l /content/drive/MyDrive/huge_dataset.txt
# !wc -l /content/huge_dataset.txt     
print(int(8927077*0.9), int(8927077*0.1))
# input ("Use that?")

# Train/test split
380253500-342228150-38025350
!head -n 8034369 /content/huge_dataset.txt > /content/train.txt
!tail -892707 /content/huge_dataset.txt > /content/valid.txt

Train our model

In [ ]:
# !pwd
# %cd /content/auto_coding
# !python3 train.py --model_select gpt2_large --with_wandb True

## Prepare data for parallel
We use custom implementation of distributed dataset. For training and evaluating we should specify file `file.list` with list of paths to txt files. All files from `file.list` will be splitted between aviable GPUs. The logic of splitting is described by the following code:

```python
shard_size = len(files) // world_size
shard_start = rank * shard_size
shard_end = (rank + 1) * shard_size
files = files[shard_start:shard_end]
```

For more details please see full code of dataset: `src.dataset_rugpt3.RuGpt3TextDataset`.

In [ ]:
!echo /content/train.txt > /content/train.list
!echo /content/valid.txt > /content/valid.list

## Train
Load model from Huggingface and finetune on essays.

This will take arount ten minutes.

In [ ]:
!mkdir log

In [ ]:
%tensorboard --logdir log

In [ ]:
%cd /content/ru-gpts
!git pull

In [ ]:
# > padded vocab (size: 50257) with 7 dummy tokens (new size: 50264)

# %tensorboard --logdir log
%cd /content
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!USE_DEEPSPEED=1 python -m torch.distributed.launch --nproc_per_node 1 ru-gpts/pretrain_gpt3.py \
  --train-data-path "train.list" \
  --test-data-path "valid.list" \
  --max-files-per-process 100 \
  --logging-dir="log" \
  --save /content/drive/MyDrive/ivolution_gpt/model \
  --load-huggingface sberbank-ai/rugpt3large_based_on_gpt2 \
  --save-interval 1000 \
  --log-interval 100 \
  --eval-interval 1000 \
  --eval-iters 100 \
  --model-parallel-size 1 \
  --num-layers 24 \
  --hidden-size 1536 \
  --num-attention-heads 16 \
  --batch-size 1 \
  --seq-length 2048 \
  --max-position-embeddings 2048 \
  --train-iters 2000 \
  --resume-dataloader \
  --distributed-backend "nccl" \
  --lr 0.00015 \
  --lr-decay-style "cosine" \
  --lr-decay-iters 3200 \
  --clip-grad 0.5 \
  --warmup .004 \
  --fp16 \
  --checkpoint-activations \
  --deepspeed-activation-checkpointing \
  --deepspeed \
  --deepspeed_config ru-gpts/src/deepspeed_config/gpt3_large_2048.json &>> /content/drive/MyDrive/gpt3_log6.log

In [ ]:
# %tensorboard --logdir log

At the end of training output should be something like this:

"-----------------------------------------------------------------------------------------

 validation loss at the end of training for test data | LM loss: 3.0002 | LM PPL: 20.090

-----------------------------------------------------------------------------------------"

## Generate

Load pretrained model from dir and generate.

In [ ]:
# !export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

# !python ru-gpts/generate_samples.py \
#   --load model/ \
#   --model-parallel-size 1 \
#   --num-layers 12 \
#   --hidden-size 768 \
#   --num-attention-heads 12 \
#   --batch-size 1 \
#   --seq-length 500 \
#   --max-position-embeddings 2048 \
#   --distributed-backend "nccl" \
#   --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
#   --no-load-optim


### Convert checkpoint to Huggingface format

In [ ]:
# !export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

# !python ru-gpts/convert2huggingface.py \
#   --load model/ \
#   --model-parallel-size 1 \
#   --num-layers 12 \
#   --hidden-size 768 \
#   --num-attention-heads 12 \
#   --max-position-embeddings 2048 \
#   --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
#   --no-load-optim \
#   --export-huggingface model_hf


#### Test load

In [ ]:
# from transformers import GPT2LMHeadModel

In [ ]:
# model = GPT2LMHeadModel.from_pretrained("model_hf")

In [ ]:
# !mkdir /content/drive/MyDrive/ivolution_gpt
# !cp -rv /content/model /content/drive/MyDrive/ivolution_gpt